# Convolution Neural Network(Theory01)

假设给予我们一张图片(28x28x3)需要判断其是不是猫,也就是说我们要做一个猫咪的分类器,那么使用我们之前所学的DNN就可以解决,但是实际上这张图片是非常小的,现代随便一张图片的大小都可能是(1000x1000x3)的图片,那么如果是这种大图片使用DNN训练的话就会发现数据过大(接近3000000),那么这么多的数据我们是没有办法进行运算的且容易导致过拟合,并且我们现在才是一张图片而言.所以我们需要使用一种方式叫**卷积神经网络(Convolution NN)**来使得参数稀疏化.

## 1.Edge detection example

卷积操作是卷积神经网络最基本的组成部分,使用边缘检测作为入门样例:

<img src="../../picture/125.png" width=500 height=500>

左图是我们需要边缘检测的图,如果我们:

- (1) 垂直的边缘检测(vertical edges),那么就如右上的显示,很明显可以看出图片垂直的部分会更加高亮(也就是像素值会更高),水平部分会暗一些(像素值底).

- (2) 水平的边缘检测(horizontal edges),那么就如右下的显示,很明显水平部分会更加高亮,垂直部分也就会暗一些.

下面我们就来看看如何操作能够达到这种效果:
### 1.1 Edge

下面的矩阵是一个(6x6)的灰度图像,构造一个(3x3)的过滤器(filter)或者叫核(kernel)进行卷积操作:

<img src="../../picture/126.png" width=500 height=500>

具体操作如下:

[1] 将filter放入(6x6)的灰度图中,计算filter与其覆盖地方的连乘求和,从而得到第一个卷积值:
>$3*1 + 1*1 + 2*1 + 0*0 + 5*0 + 7*0 + 1*(-1) + 8*(-1) + 2*(-1) = -5$
><img src="../../picture/127.png" width=500 height=500>

[2] 将filter左移一个像素值,继续进行如上操作,得到第二个卷机值:
>$0*1+5*1+7*1+1*0+8*0+2*0+2*-1+9*-1+5*-1=-4$
><img src="../../picture/128.png" width=500 height=500>

[3] 继续右移,当移到最后一片区域的时候,整体下移一个像素:
>$1*1+2*1+0*1+5*0+7*0+1*0+8*-1+2*-1+3*-1=-10$
><img src="../../picture/129.png" width=500 height=500>

[4] 继续重复上述操作,知道整个(6x6)的灰度图滑动完毕:
><img src="../../picture/130.png" width=500 height=500>

这就是CNN中的卷积操作也称之为局部链接.

**局部连接**:在处理图像这样的高维度输入时,让每个神经元都与前一层中的所有神经元进行全连接是不现实的(因为数据量太大).相反,我们让每个神经元只与输入数据的一个局部区域连接.该连接的空间大小叫做神经元的感受野(receptive field).


这里附上一张动态图:

<img src="../../picture/158.gif" width=500 height=500>

**Ps:**

- 灰度图:没有RGB通道,维度为2.
- 上述的卷积操作,并不是真正意义上数学中的卷积操作,数学意义中的卷积操作是需要将卷积核镜像一次,而这里严格意义上来说应该叫"互相关".

那么为什么这种卷积操作可以用于垂直检测呢?主要还是因为该卷积核的原因.为了说明方便,我们选用下面这张灰度图:

<img src="../../picture/131.png" width=300 height=300>

这张灰度图有一个很明显的地方,其左半边由于像素值较高,所以高亮,右边像素值较低所以较暗也就是如下显示:

<img src="../../picture/132.png" width=100 height=100>

可以看到有明显的边界(图中蓝色的部分),那么我们使用上述的卷积核进行卷积操作即:
<img src="../../picture/133.png" width=500 height=500>

那么对于该次卷积操作而言:
- (1) 卷积核是一个左边较亮,中间微暗,右边较暗的(3x3)像素组成.
- (2) 通过卷积过后我们可以发现在中间垂直的部分(也就是下面两个**红色圈起来的部分**)会变得更亮,这样也就做到的垂直部分的边缘检测,如果不好理解,我们拿回这张图:
    - <img src="../../picture/125.png" width=500 height=500>
    - 可以看到图中红色的竖线部分,如果你仔细看,你会发现检测出来的垂直边缘是高亮的,这就应对了上面这个例子最后高亮的部分.
    - 另外在原图中也可以发现,检测出来的垂直部分两侧的"亮和暗"也有高度过渡.
    
[PDF](../../PDFS/C4W1L02-EdgeDetectionExample.pdf)

## 2. More edge detection

现在我们将区分正边和负边,实际上就是“亮到暗”或者“暗到亮”的区别也就是边缘过渡.

如果我们将上面的灰度图(6x6)的反转一下(左边较暗,右边较亮)并且使用垂直卷积核:

<img src="../../picture/134.png" width=500 height=500>

可以看到经过卷积之后得到的结果就是两边较亮,中间较暗.之前我们说过当数据集不够的时候,我们可以将图片进行反转来得到更多的数据集,但是如果将反转的图片进行卷积操作,那么就会改变原来真实的结果,当然这样做有好处也有不好的地方:

(1) 好处:可以增加一定范围内的噪声,防止模型过拟合

(2) 坏处:反转后的图片进行卷积得到的边缘已经不是真正的边缘,所以在一些情况下是"污染".

对于这种情况,我们可以对输出矩阵取绝对值.

无论如何,这种卷积核能够帮我们识别过渡区域,也就是从“暗到亮”或者从“亮到暗”的区别.

### 2.1 More Edges

除了上面的垂直卷积核,还有很多其他的卷积核:

[1] 水平卷积核(Horizontal),也就是垂直卷积核旋转90°.

$Horizontal=\begin{bmatrix}
1 &1  &1 \\ 
 0&0  &0 \\ 
 -1& -1 &-1 
\end{bmatrix}$

[2] sobel filter

$sobel\;filter=\begin{bmatrix}
1 &0  &-1 \\ 
 2&0  &-2 \\ 
 1& 0 &-1 
\end{bmatrix}$

[3] Scharr filter

$Scharr\;filter=\begin{bmatrix}
3 &0  &-3 \\ 
10&0  &-10 \\ 
 3& 0 &-3 
\end{bmatrix}$

实际上你不一定要使用上面的几种卷积核,更一般的我们会选择去"学习"这个卷积核来达到更高的效果,这里的"学习"是指反向传播,即:

[4] Learning conv(3x3)

$Conv\;Weights=\begin{bmatrix}
w_1 &w_2  &w_3 \\ 
w_4&w_5&w_6 \\ 
 w_7& w_8 &w_9 
\end{bmatrix}$

这样通过神经网络进行学习的卷积核可以检测出更一般的角度,比如45,75,或者其他任意角度.

[PDF](../PDFS/C4W1L03-MoreEdgeDetection.pdf)

## 3. Padding

在卷积神经网络中,最基本的一项就是**padding.**

之前的例子我们已经知道如果使用(6x6)的原始图像卷积核为(3x3)那么得到的是一个(4x4)的图像,也就是满足一下数学关系:

$(n\times n)"*"(f\times f)=(n-f+1 \times n-f+1)$

以(6x6)为例子则是:$6-3+1=4$

那么这样做的话实际上会有两个缺点:

[1] 如果你们网络很深,那么每次卷积操作后实际上图片就会变小,那么理论上在一定重复次数过后,那么很有可能会变成(1x1)的情况,这不是我们想要的.

[2] 注意下面这张图:

><img src="../../picture/135.png" width=300 height=300>

> 可以看你在左上角的绿色部分,其只能给予卷积核一次特征信息,那么多次经过卷积核处理后,可以说左上角的特征信息基本会大幅度的减少.而中间部分(紫色)则会多次给予卷积核信息,这样就可能会产生过多的重复信息.

为了解决这样的问题,我们引入**padding**,具体操作如下(假设padding=1):

我们会在原图上扩增一层为0的像素边:

<img src="../../picture/136.png" width=300 height=300>

这样做了之后,我们就可以解决上述的两个问题:

[1] 图像缩小的问题:

- 由于现在我们不在是(6x6)的大小而是(8x8)的大小,那么经过卷积后得到的大小依然是(8-3+1=6).

[2] 某些角落像素值给予卷积核特征信息过少:

- 我们经过padding后,可以发现现在左上角的绿色像素点不在只是一次提供给卷积核信息,而是像紫色部分一样提供多次.

那么现在加入padding后,我们的结果**计算公式变为**:

$(n+2p-f+1) \times (n+2p-f+1)$

比如我们使用(8x8)的示例:

$(8-3+1=6).$

所以只要我们想保证在卷积操作后,图像大小不发生任何变化:

$(n+2p-f+1)=n \Rightarrow p=\frac{f-1}{2}$

### 3.1 Choose Padding

通常Padding的选取有两种:

(1) Valid: 不填充,也就是padding=0,那么卷积操作之后图像会变小.

(2) Same: 填充输入与输出相同,也就是满足$p=\frac{f-1}{2}$

**Ps:**

在Computer Vision中卷积核通常取奇数,原因有两个:

(1) 取奇数能够保证卷积核自然的填充完整,而不会像偶数一样(发生小数)填充不完整.

(2) 在某些时候,奇数的卷积核能够保证一个中心点的位置.方便寻找与运算.

[PDF](../PDFS/C4W1L04-Padding.pdf)

## 4. Strided convolutions

在卷积中,卷积步也是另一个比较基础的操作.

假设我们现在对一个(7x7)的,卷积核为(3x3)做卷积操作亦即:


<img src="../../picture/137.png" width=500 height=500>


现在我们将步长(stride)设置为2,则过程如下:

> [1] <img src="../../picture/138.png" width=500 height=500>

> [2] <img src="../../picture/139.png" width=500 height=500>

> [3] <img src="../../picture/140.png" width=500 height=500>

> [4] ......

> [5] <img src="../../picture/141.png" width=500 height=500>

那么在加入步长了之后,我们的**公式**就为:

Given:$n \times n\;image, f\times f\;filter ,padding\;p, stride\;s$

$\left \lfloor \frac{n+2p-f}{s} +1 \right \rfloor \times \left \lfloor \frac{n+2p-f}{s} +1 \right \rfloor $

以(7x7)为例:

$\frac{7+0-3}{2}+1=3$

这里的$\lfloor \rfloor$表示向下取整的意思,因为如果设置步长不当,容易出现以下情况:

<img src="../../picture/142.png" width=300 height=300>

也就是说,原图无法支撑卷积核完整卷积.这样的情况,一般按照**惯例**我们需要舍弃不能完全卷积(扫描)的部分,以避免维度不是整数的情况,举一个例子:

如果是一个(8x8)的图片,padding=0,stride=2,那么结果是$(8+0-3) / 2 + 1=3.5$,这种情况下我们只能滑动(扫描三次(紫,绿,橙)):

<img src="../../picture/143.png" width=300 height=300>

我们需要放弃红色部分的像素值或者第一列.

**Ps:**

- 在数学上的卷积操作是需要将卷积核进行一次镜像的:

- $\begin{bmatrix}
3 &4  &5 \\ 
 1& 0 &2 \\ 
 -1& 9 &7 
\end{bmatrix}\Rightarrow\begin{bmatrix}
7 &9  &-1 \\ 
 2& 0 &1 \\ 
 5& 4&3
\end{bmatrix}$

也就是rot180°.

- 根据以往的经验,在卷积神经网络中,实际上你并不需要此步骤去搭建卷积神经网络,因为我们的初始参数是随机给定的,在后面也需要通过反向传播来更新参数

- 很多的深度学习论文中也没有将卷积核镜像化,他们直接管"互相关"叫卷积.

[PDF](../PDFS/C4W1L05-StridedConv.pdf)

## 5. 3D Convolutions

上面我们已经知道了如何对2D的灰度图做卷积操作,现在我们来看看对于3D的彩色图是如何做卷积操作的如下所示:

<img src="../../picture/144.png" width=500 height=500>

这里需要注意的是:

- (1) 我们的原图采用的是(heights,widths,\#channels)的形式,在cs231n中采用的是(heights,widths,depth).

- (2) 在做3D卷积的时候,一定要保证卷积核$(f\times f \times n_c)$,也就是说卷积核的depth要与**原图是一致的.**

<span style="color:orange">接下来我们将该卷积核放在原图上滑动,对于每个通道都会产生9个卷积值,总共27个值,再将这27个值累和,得到输出(4x4)中的第一个值.然后向右移动一个步长继续得到第二个27值并求和,得到输出(4x4)中的第二个值,一次类推到最后一次滑动.</span>

<img src="../../picture/CONV.gif" width=300 height=300>

也就是说:

将卷积核中每一个通道的数字与原图中每一个通道的数字相乘再累和就得到输出结果中的第一个值,也就是说RGB,三层分别操作(类似于2D中的单层操作三次)

例子中是:R中3个值乘上卷积核中第一层三个值+G中三个值乘上卷积核中第二层三个值+B中三个值乘上卷积核中底三层的3个值,也就是27个值相加得到输出的第一个值.

**立体卷积的好处是:**

如果你指向检测R通道(比如垂直检测),那么你只需要在第一层上面写入,其他层都为0,那么该卷积核就只会检测R通道的垂直边缘,同样GB也是如此.

<img src="../../picture/145.png" width=400 height=400>

如果你想整体RGB都检测,那么可以在卷积核的三层都放图垂直检测数据,就可以检测任意颜色通道的边界:

<img src="../../picture/146.png" width=400 height=400>

**PS**

- 我们现在使用3D卷积出来的结果是一个2D的,即:输入的$(6\times 6\times 3) "*" (3\times 3\times 3)$最后得到的是2-D的$(4\times 4)$

- 另外在一些框架下,比如TF,Pytorch,Keras,这种3D卷积统称为"2DCONV".详情查看案例.

### 5.1 More Filter

如果我们想既检测垂直又检测水平或者45°,75°等等,该怎么办呢?也就是我们想同时使用**多个卷积核**检测不同情况下的边缘.

比如我们既想检测垂直边缘,又想检测垂直边缘,事实上我们可以使用两次卷积核,并且将**结果重叠**.


<img src="../../picture/147.png" width=400 height=400>


对于黄色的卷积核(垂直检测),操作与上面单个3D卷积核完全一样,产生一个(4x4)的结果,然后再使用橙色的卷积(水平检测)核操作一次得到一个(4x4)的卷积核,然后再将橙色的结果叠加在黄色卷积核的结果后面.

这样我们就既能检测垂直,又能检测水平,那么如果使用"反向传播"进行学习,我们就可以同时学习到任意角度的特征了.

下面附上cs231n中的动态图:

In [1]:
from IPython.display import IFrame
IFrame('http://cs231n.github.io/assets/conv-demo/index.html', width='100%', height='700px')

所以我们的**公式符号**更改为:

$n\times n\times n_{c} \ *\ f\times f\times n_{c} \Rightarrow \left \lfloor \frac{n+2p-f}{s} +1 \right \rfloor \times \left \lfloor \frac{n+2p-f}{s} +1 \right \rfloor \times n_{c}^{'}$

$ n_{c}^{'}:Number\;of\;filter$,也就是卷积核的个数(最后结果的深度(depth))

以(6x6x3)和卷积核(3x3x2)为例: $(6\times 6\times 3 "*" 3\times 3\times 2 \Rightarrow 4\times 4\times 2)$

<span style="color:orange">这样你就可以使用多个卷积核同时检测特征数了,输出的结果第三维度的数字就是特征的量.</span>

[PDF](../PDFS/C4W1L06-ConvolutionsOverVolumes.pdf)

## 6. One layer of a convolutional network

下面我们来看看卷积操作是如何在一层神经网络中使用的

上面我们已经知道对于RGB原图可以同时经过多个卷积核处理,并得到一个输出且输出的结果,其中第三维度是卷积核的数量.

<img src="../../picture/148.png" width=400 height=400>

现在我们只需要将输出的结果做:

(1) 线性:weights1(4x4) + bias1=z1,激活函数:Relu(z1)=a1

(2) 线性:weights2(4x4) + bias2=z2,激活函数:Relu(z2)=a2

<img src="../../picture/149.png" width=400 height=400>

那么我们就完成了一个从(6x6x3)到(4x4x2)的输出:

<img src="../../picture/150.png" width=400 height=400>
这就是卷积神经网络的一层.

**Note:**如果有看不懂的可以查看cs231n的图

使用数学符号表达即为:

$z^{[1]} = w^{[1]}a^{[0]}+b^{[1]}$

$a^{[1]} = g^{[1]}(z^{[1]})$

$w^{[1]} shape : 3\times 3\times 3\times 2$

$b^{[1]} shape:\;Relu\;Number\;1\times 1\times 2 $

$a^{[1]} shape:\;4\times 4\times 2  = z^{[1]}$

$a^{[0]}:$ Input layer

### 6.1 Summary of notation

现在我们来看看更一般的情况下($l$层)

卷积核的个数: $f^{[l]} = filter\;size$ 

padding长度: $p^{[l]} = padding$

滑动步长: $s^{[l]} = stride$

卷积核个数(Real Number): $n_{c}^{[l]} = number\ of filters$

$n_{c}^{[0]}$(Real Number):表示为图片的通道数($\#Channels$)

假设我们有以下的输入:
> [1] 在上述例子中,我们都是使用宽高一样的图片,为了更加一般化,我们将设置$h:height,w:width$

>[2] $Input: n_{h}^{[l-1]}\times n_{w}^{[l-1]}\times n_{c}^{[l-1]}$

> [3]因为我们知道卷积操作后结果的深度就是卷积核个数,所以结果为:

> [3.1] $OutPut:n_{h}^{[l]}\times n_{w}^{[l]}\times n_{c}^{[l]}$

> [3.2] $OutPut\; Size: \left \lfloor \frac{n_{h}^{[l-1]}+2p^{[l]}-f^{[l]}_{h}}{s^{[l]}} +1 \right \rfloor \times \left \lfloor \frac{n_{w}^{[l-1]}+2p^{[l]}-f^{[l]}_{w}}{s^{[l]}} +1 \right \rfloor \times n_{c}^{[l]}$


  
那么对于每一层的卷积核而言:

- Each filter is: $f^{[l]}\times f^{[l]}\times n_{c}^{[l-1]}$

对于每一层的激活函数而言:

- Activations$(a^{[l]}=activate(z^{[l]})=activate(a^{[l-1]}\cdot w^{[l]}+b^{[l]}))$:
    - $n_{h}^{[l]}\times n_{w}^{[l]}\times n_{c}^{[l]} $ 
    - if m example,then$ A^{[l]} :m\times n_{h}^{[l]}\times n_{w}^{[l]}\times n_{c}^{[l]} $

对于每一层的$Weights$而言:

- weights shape:$f^{[l]}\times f^{[l]}\times n_{c}^{[l-1]}\times n_{c}^{[l]}$

对每一层的$bias$而言:
- bias shape:$n_{c}^{[l]}$

- In code:$1 \times 1\times n_{c}^{[l]} $


**Ps:**

另外在其他一些代码或者框架中可能把通道放在首位进行排放:

$A^{[l]} :m\times n_{c}^{[l]}\times n_{h}^{[l]}\times n_{w}^{[l]}$如果是这种,那么整体的卷积神经网络的通道顺序记得都要变.

在Tensorflow中可以通过参数```data_format```自定义选择:

<img src="../../picture/151.png" width=500 height=500>

在Pytorch中是通道优先:

<img src="../../picture/152.png" width=500 height=500>

实际上哪种都可以,依照自己选择就行.

### 6.2 Number of parameters in one layer

现在我们来看看卷积神经网络中需要学习的参数有多少(在一层中):

之前我们使用的卷积核是(3x3x3x2)的,那么对于一个卷积核而言,需要学习的参数为:3x3x3=27个,再加上一个偏置b,所以单个卷积核我们需要学习的参数是28个,那么对于两个卷积核就是(2x28=56个参数),那么假设有10个卷积核就是(10x28=280个参数).

**请注意:**<span style="color:orange">无论图片的输入为多大,如果是10个卷积核那么参数都是280,因为参数只与卷积核的定义有关与图片大小无关.所以这就是卷积神经网络的一个特征叫做"避免过拟合".</span>

## 7. Example ConvNet

下面我们来看一下简单的的卷积神经网络(single sample).

<img src="../../picture/153.png" width=700 height=700>

[1] 对于输出层(Input):

- $n_{H}^{[0]} = n_{W}^{[0]} = 39$

- $n_{c}^{[0]} = 3$

[2] CONV1:使用$f^{[1]} = 3\times 3\times 3,s^{[1]} = 1,p^{[1]} = 0,10\ filters$,卷积核简写为$f^{[1]} = 3$,得到结果:

- $n_{H}^{[1]} = n_{W}^{[1]} = \frac{39+2*0-3}{1}+1=37$

- 由于我们使用的是10个卷积核,所以 $n_{c}^{[1]} = 10$

[3] CONV2:使用$f^{[2]} = 5,s^{[2]} = 2,p^{[2]} = 0,20\ filters$

- $n_{H}^{[2]} = n_{W}^{[2]} = \frac{37+2*0-5}{2}+1=17$

- 我们使用20个卷积核,所以 $n_{c}^{[2]} = 20$

[3] CONV3:$f^{[3]} = 5,s^{[3]} = 2,p^{[3]} = 0,40\ filters$

- $n_{H}^{[3]} = n_{W}^{[3]} = \frac{17+2*0-5}{2}+1=7$

- 我们使用40个卷积核,所以 $n_{c}^{[3]} = 40$

接下去我们将CONV3得到的结果展开得到(7x7x40=1960,1)的矩阵,我们称之为:全连接层(Fully Connected:FC)

[4] Output:将Logistics或者Softmax应用于FC即可得到我们的预测值.

**Ps:**

(1) 实际上FC就相当于我们之前所搭建的NN.

(2) CONV称之为卷积层,可以看到随着网络深度的增加,如果padding=0,那么图片的heights and widths会变小,但是channels会增加.

(3) <span style="color:orange">我们的Input数据量为(39x39x3=4563),经过卷积之后数据量为(7x7x40=1960),很明显训练的数据量变少了,这就是CNN的数据稀疏化,能够有效的预防过拟合</span>


实际上典型的CNN通常只有三层:

[1] Convolution (卷积层)就是上面的例子 

[2] Pooling (池化层) (POOL)

[3] Fully Connected （全连接层）


下面我们就来看看池化层(Pooling)

[PDF](../PDFS/C4W1L08-SimpleCNNExample.pdf)

## 8 Pooling layers

下面我们来看看池化层,池化层(汇集层)一般分为两种:1.Max Pooling,2.Average Pooling

### 8.1 Max Pooling

现在假设我们输入的是一个(4x4),Max Pooling的操作如下:

[1] 类似与卷积核,取filters=2x2,stride=2,然后取滑动窗口内的最大值:

<img src="../../picture/154.png" width=700 height=700>

这样就得到了Max Pooling的结果,这里有两个超级参数**Filters,Stride**通常我们取$filter=2,stride=2$,也就意味着经过Max Polling后大小减小为原来的一半.

那么对于Max Polling的直观理解是:

我们把这个4x4的输入看做是某些特征的集合,那么Max Polling做的就是提取窗口内特征值**最大的像素点**,因为特征值较大的像素点往往可以提供更多的信息,例如我们想象在黑暗中有一点光源,那么这个光源就是这一片黑暗中最重要的一点,所以我们应该保留下来,当然这不一定是准确的解释,因为某些情况下数值较小的特征也可能是我们的目标,这样说只是为了一个方便的理解.总而言之,Max Polling的目的就是为了提取图像中比较重要的特征,而且Max Polling在过去的实验中取得的效果都是很优秀的.

**Ps:**

- max polling 是不需要学习参数的,也就是说不需要反向传播学习,因为一旦filters和stride定下来了之后,它就是一个固定的运算,只是单纯的获取该窗口下所有特征值中最大的那个特征值.

### 8.1.1 More Max Polling

假设我们要在一个(5x5)做Max Polling:


<img src="../../picture/MaxPolling.gif" width=700 height=700>

可以看到比较大的特征值就都出来了.

我们这里使用$f=3x3,s=1$,实际上:

$\left \lfloor \frac{n_{h}^{[l-1]}+2p^{[l]}-f^{[l]}}{s^{[l]}} +1 \right \rfloor \times \left \lfloor \frac{n_{w}^{[l-1]}+2p^{[l]}-f^{[l]}}{s^{[l]}} +1 \right \rfloor \times n_{c}^{[l]}$

同样适用于Polling layers.

比如我们这里的(5x5):$\frac{5-3}{1}+1=3$的输出结果.



**需要注意的是,一般在Polling layer中我们不会使用Padding.**

<span style="color:orange">同样,如果是3D的例子,我们就对RGB三个通道各做一次polling就可以了,得到就是一个深度(depth=3)的结果.</span>

### 8.2 Average Polling

平均池化也是池化的一种,但是很少用到,操作和Max Polling一样,只不过从滑动窗口获取最大值变为获取平均值.

<img src="../../picture/155.png" width=700 height=700>

### 8.3 Summary of Pooling

(1) 常用的f = 2,s = 2,相当于高度和宽度各减少了一半,也有使用f = 3,s =2的情况,建议多看看别人的论文,看看别人是怎么设置的.

(2) 我们很少在polling中使用padding,padding = 0

(3) 因为我们很少在Pooling中使用padding,所以我们的输出大小如下:

- $ \left \lfloor \frac{n_{h}^{[l-1]}-f^{[l]}}{s^{[l]}} +1 \right \rfloor \times \left \lfloor \frac{n_{w}^{[l-1]}-f^{[l]}}{s^{[l]}} +1 \right \rfloor \times n_{c}^{[l]}$

(4) 在一般情况下:pooling输出的通道数一定与输入的是一样的,因为我们要对每一层通道都做polling.

(5) 超级参数$f_{pooling},stride_{pooling}$是不需要学习的,是网络的一个静态属性.

(6) 在实践中,Max pool通常只有两种形式:1.$f=3,s=2$也叫重叠汇聚(overlapping),但是更加常用的还是$f = 2,s = 2$.对更大感受野进行pool需要的pool尺寸也更大,但是往往对网络有破坏性.

[PDF](../PDFS/C4W1L09-PoolingLayers.pdf)

## 9.  Convolutional neural network example(single sample)

这里我们使用LeNet-5(Ng版本)网络完成手写数字识别的流程.

<img src="../../picture/156.png" width=700 height=700>

在这里,我们将CONV和Pool作为layer,但是也有将pool单独作为一层的,这样方便手写反向传播计算,这个没有明确定义问题,通常人们在计算神经层数的时候,只是统计含有权重和参数的层数,所以这里我们将CONV和Poo作为一层,因为Pool中没有参数或权重,很多框架如Tensorflow,Pytorch都是将poling独立出来作为一个函数.

CNN网络解释:

[1] CONV1:我们采用$f=5,s=1,filters=6$,应用了线性:$z^{[1]}=a^{[0]}\cdot w^{[1]}+b^{[1]}$,非线性函数比如ReLu:$a^{[1]}=ReLu(z^{[1]})$

[1.1] Pool1:接下去我们使用max polling$(f = 2,s = 2,No\ padding)$,得到layer1.

[2] CONV2:$f=5,s=1$应用了线性:$z^{[2]}=a^{[1]}\cdot w^{[2]}+b^{[2]}$,非线性函数比如ReLu:$a^{[2]}=ReLu(z^{[2]})$

[2.2] Pool2:$f=2,s=2$最终得到layer2.

[2.2.1] 归一层:将Pool2得到的结果展开为$a^{[2]}=(400,1)$的向量开始进入FC

[3] FC3:初始化权重$w^{[3]}=(120,400)$,$b^{[3]}=(120,1)$应用了线性:$z^{[3]}=w^{[3]}\cdot a^{[2]}_{flatten}+b^{[3]}$,,非线性函数比如ReLu:$a^{[3]}=ReLu(z^{[3]})$

[4] FC4:初始化权重$w^{[4]}=(84,120)$,$b^{[4]}=(84,1)$应用了线性:$z^{[4]}=w^{[4]}\cdot a^{[3]}_{flatten}+b^{[4]}$,,非线性函数比如ReLu:$a^{[4]}=ReLu(z^{[4]})$

[5] Softmax:初始化权重$w^{[5]}=(10,84)$,$b^{[5]}=(10,1)$应用了线性:$z^{[5]}= w^{[5]}\cdot a^{[4]}_{flatten}+b^{[5]}$,,非线性函数比如Softmax:$a^{[5]}=Softmax(z^{[5]})$

**Notic**:

(1) 这里面有很多的超级参数,常规的做法是尽量不要自己设置超级参数,应该多从文献中去查看,看看别人框架中的参数是否合适自己.

(2) 另外随着神经网络的深度增加,高度和宽度通常都会减少,只有输出信息通道数Channels(或者特征层数)在增大.

(3) 卷积神经网络最常见的模式就是上面这种,CONV1-POOL1-CONV2-POOL2-FC3-FC4-SOFTMAX

(4) 在卷积神经网络的结构中,提出了很多不同类型的归一化层,有时候是为了实现在生物大脑中观测到的抑制机制.但是这些层渐渐都不再流行,因为实践证明它们的效果即使存在,也是极其有限的.对于不同类型的归一化层,可以看看Alex Krizhevsky的关于[cuda-convnet library API](https://code.google.com/p/cuda-convnet/wiki/LayerParams#Local_response_normalization_layer_(same_map))的讨论。

(5) 全连接层和卷积层之间唯一的不同就是卷积层中的神经元只与输入数据中的一个局部区域连接,并且在卷积列中的神经元共享参数.
- 对于任一个卷积层,都存在一个能实现和它一样的前向传播函数的全连接层.权重矩阵是一个巨大的矩阵,除了某些特定块(这是因为有局部连接),其余部分都是零.而在其中大部分块中,元素都是相等的(因为参数共享).

- 相反,任何全连接层都可以被转化为卷积层.比如一个$K=4096$的全连接层,输入数据体的尺寸是$7\times 7\times 512$,这个全连接层可以被等效地看做一个$f=7,p=0,s=1,K=4096$的卷积层,换句话说,就是将滤波器的尺寸设置为和输入数据体的尺寸一致了因为只有一个单独的深度列覆盖并滑过输入数据体,所以输出将变成$1\times 1\times 4096$.


### 9.1 Parameters in Example

下面我们来看看该例子中的参数情况:

<img src="../../picture/157.png" width=700 height=700>

从图中我们可以看出,卷积层的参数较少,主要的参数还是在全连接层.

需要注意的是,我们的Activation Size下降的不能过快,否则会影响网络的性能.

**PS:**

像这种传统的将层按照线性进行排列的方法已经受到了挑战,挑战来自谷歌的Inception结构和微软亚洲研究院的残差网络(Residual Net)结构.这两个网络的特征更加复杂,连接结构也不同.

[PDF](../PDFS/C4W1L10-CNNExample.pdf)

## 10. Why CNN?

那么我们为什么要使用CNN呢?


### 10.1 参数共享

<img src="../../picture/159.png" width=700 height=700>



参数共享就是:给一张图片用卷积核去滑动,那么由于图片是经过一个卷积核扫过,而卷积核中的权重值又不会变,所以称之为参数共享.


下述来自cs231n:

在[Krizhevsky](http://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf)构架赢得了2012年的ImageNet挑战,其输入图像的尺寸是[227x227x3].在第一个卷积层,神经元使用的感受野尺寸$f=11,s=4,p=0$,因为(227-11)/4+1=55,卷积层的深度$K=96$则卷积层的输出数据体尺寸为[55x55x96].如果我们将[55x55x96]中的神经元(55x55x96=290,400)都使用不同的卷积核,也就是说每一次滑动的[11x11x3]中的权重参数**都不一样**(注意:我们之前所做的卷积操作中卷积核的权重参数是一样的),那么最终将会产生55x55x96=290,400个神经元和每个神经元有11x11x3=364个参数和1个偏差,最终总体参数为290400x364=105,705,600个参数.显然,这样参数就过多了.

那么作一个合理的假设:如果一个特征在计算某个空间位置(x,y)的时候有用,那么它在计算另一个不同位置(x2,y2)的时候也有用.基于这个假设,可以显著地减少参数数量.换言之,就是将深度维度上一个单独的2维切片看做深度切片(depth slice),比如一个数据体尺寸为[55x55x96]的就有96个深度切片,每个尺寸为[55x55].在每个深度切片上的神经元都使用同样的权重和偏差.在这样的参数共享下,例子中的第一个卷积层就只有96个不同的权重集了,一个权重集对应一个深度切片,共有96x11x11x3=34,848个不同的权重,或34,944个参数（+96个偏差）.在每个深度切片中的55x55个权重使用的都是同样的参数.也就是说为什么,我们在之前所做的卷积操作:每次滑动都使用同一个卷积核(感受野).

这样的假设是合理的因为:如果在图像某些地方探测到一个水平的边界,那么在其他一些地方也会有水平边界.所以在卷积层的输出数据体的55x55个不同位置中,就没有必要重新学习去探测一个水平边界了.


**注意:**有时候参数共享假设可能不适用.特别是当CNN的输入图像有一些特别的中心结构时,我们可以想想,在一侧应该学习的功能会与另一侧完全不同.一个实际的例子就是当输入是人脸的时候,人脸总是处于图像的中心.那么人脸左边学到的内容与右边学到的内容是对称的,而不是相似的(比如鼻子下部的👃,是左右对称的),所以我们就更加希望在不同的位置都被学习.在这种情况下通常我们会放松参数共享的方案,转而简单地称呼该层为局部连接层(Locally-Connected Layer).写到这里的时候,我个人感觉如果没有参数共享的假设,那么我们可以令每一次滑动卷积核中的权重参数都不同.


**Ps:**

在原论文中,说的输入图像尺寸是224x224,这是肯定错误的,因为(224-11)/4+1的结果不是整数.这件事在卷积神经网络的历史上让很多人迷惑,而这个错误到底是怎么发生的没人知道.我的猜测是Alex忘记在论文中指出自己使用了尺寸为3的额外的零填充.并且第一层CONV的神经元数也不对.

### 10.2 稀疏链接

<img src="../../picture/160.png" width=700 height=700>

图中输出结果左上角的0(绿色)是3x3的卷积核计算出的,所以它只依赖于这个3x3的输入单元格.因此这个0只与原图36个特征中的9个相连接(输入的绿色部分),而且其他区域的特征值都不会对输出产生影响,这就是稀疏链接的概念.



[PDF](../PDFS/C4W1L11-WhyConvs.pdf)

### 10.3 如何理解卷积网络获得的位移、尺度、形变不变性

卷积网络的核心思想是将:局部感受野(卷积核),权值共享(或者权值复制,参数共享)以及时间或空间亚采样这三种结构思想结合起来获得了某种程度的位移、尺度、形变不变性.这三种思想也可叫局部连接(local connections)权值共享(sharedweight),池化(pooling).

[1] 位移不变性:图像在平移后在特征图上的表示也是同样平移的,这就使图像拥有了一定的平移不变性.

[2] 尺度不变性:同样的,pooling(以MAX Pooling为例),对局部感受野取其极大值,如果图像在尺度上发生了变化,有一定概率在尺度变化后对应的感受野取到的极大值不变,这样就可以使特征图不变,同样也增加了一定的平移不变性,pooling就相当于实现了对这种设计出来的变化的不变性.

- 平移不变性:图像经历了一个小的平移之后,依然会产生相同的(池化的)特征.比如图片中有猫,那么向右移动两个像素点,那么图片中的猫应该也是清晰可见的,因为卷积核移动几个像素点,这张图片依然具有非常相似的特性,就像你把图片的所有像素点都偏移一点点,那么这张图的特性应该都是还在的.池化(max pooling)提取到的最大特征值依然是原来的特征,继而全连接层中特征应该还是被保留的,所以我们说平移不变性是相对于全连接层而言的.

[3] 对于形状不变性,实际上,在图像识别中,**重要的不是显著特征的绝对位置而是相对的位置**,所以为了避免把过多的位置信息编码进去,卷积和池化的操作都可以对局部的纹理进行模糊化,这样也就使图像有了一定的形状的不变性.

**ps:**

关于平移不变性的讨论:

在这片paper中[Why do deep convolutional networks generalize so poorly to small image transformations?](https://link.zhihu.com/?target=https%3A//arxiv.org/pdf/1805.12177.pdf)

知乎讨论区域:[证伪:CNN中的图片平移不变性](https://zhuanlan.zhihu.com/p/38024868)

论文却揭示了这样一个现实:当图像在当前平面上平移几个像素后,现代CNN（如VGG16、ResNet50和InceptionResNetV2）的输出会发生巨大改变,而且图像越小,网络的识别性能越差;同时,网络的深度也会影响它的错误率.

论文通过研究表明,产生这个现象的主因是现代CNN体系结构没有遵循经典采样定理,无法保证通用性,而常用图像数据集的统计偏差也会使CNN无法学会其中的平移不变性.

<img src="../../picture/161.png" width=700 height=700>



在上图中,左侧图像是模型的输入,右侧折线图是模型评分,使用的模型是InceptionResNet-V2 CNN.可以发现,作者在这里分别对图像做了平移、放大和微小形变.在最上方的输入中,他们只是将图像从左到右依次下移了一像素,就使模型评分出现了剧烈的波动;在中间的输入中,图像被依次放大,模型的评分也经历了直线下降和直线上升;而对于最下方的输入,这三张图是从视频中选取的连续帧,它们在人类眼中是其是自然运动姿态,但在CNN“眼中”却很不一样,模型评分同样遭遇跌入低谷.

本人对该论文的讨论持中立态度,评论中有说:应该先定位.但是我想该论文本身就不在"先定位"的结果下,只是单纯从"CNN平移不变性"来做探讨.

**Reference:**

[1] [CNN十大问](http://www.voidcn.com/article/p-ahduptmb-dx.html)

[2] [证伪:CNN中的图片平移不变性](https://zhuanlan.zhihu.com/p/38024868)

## 11. 层的尺寸设置规律

现在为止,我们都没有提及卷积神经网络中每层的超参数的使用.现在先介绍设置结构尺寸的一般性规则,然后根据这些规则进行讨论:

### 11.1 输入层

一般输入层(包含图像的)应该能被2整除很多次.常用数字包括32(比如CIFAR-10),64,96(比如STL-10)或224(比如ImageNet卷积神经网络),384和512.

### 11.2 卷积层

应该使用小尺寸的卷积核(比如3x3,5x5),**使用步长$s=1$**,还有一点非常重要,就是要对输入的数据进行padding,这样卷积层就不会改变输入数据空间上维度的尺寸($(n+2p-f+1)=n \Rightarrow p=\frac{f-1}{2}$).如果必须使用更大的滤波器尺寸(比如7x7之类),通常只用在第一个面对原始图像的卷积层上.

为什么在卷积层使用1的步长?在实际应用中,更小的步长效果更好.步长为1可以让空间维度的降采样全部由(池化层)汇聚层负责,卷积层只负责对输入数据体的深度进行变换.

### 11.3 池化层(汇聚层)

负责对输入数据的空间维度进行降采样,最常用的设置是用用$2\times 2$感受野即$F=2$的最大值汇聚,步长为2($s=2$),注意这一操作将会把输入数据中75%的激活数据丢弃(因为每一个$2\times2$都要丢弃$\frac{3}{4}$的特征值).一个不那么常用的设置是使用$3\times 3$的感受野,步长为2,汇聚的感受野尺寸很少有超过3的,因为汇聚操作过于激烈,易造成数据信息丢失,这通常会导致算法性能变差.

很多人不喜欢汇聚操作,认为可以不使用它.比如在[Striving for Simplicity: The All Convolutional Net](https://arxiv.org/pdf/1412.6806.pdf)一文中,提出使用一种只有重复的卷积层组成的结构,抛弃汇聚层.通过在卷积层中使用更大的步长来降低数据体的尺寸.有发现认为,在训练一个良好的生成模型时,弃用汇聚层也是很重要的.比如变化自编码器(VAEs:variational autoencoders)和生成性对抗网络(GANs:generative adversarial networks).现在看起来,未来的卷积网络结构中,可能会很少使用甚至不使用汇聚层.

[下采样(降采样)与上采样](https://blog.csdn.net/majinlei121/article/details/46742339)

### 11.4 妥协

因为内存限制所做的妥协:在某些案例(尤其是早期的卷积神经网络结构)中,基于前面的各种规则,内存的使用量迅速飙升.例如,使用64个尺寸为3x3的滤波器对224x224x3的图像进行卷积,padding为1,得到的激活数据体尺寸是[224x224x64].这个数量就是一千万的激活数据,或者就是72MB的内存(每张图就是这么多,激活函数和梯度都是).因为GPU通常因为内存导致性能瓶颈,所以做出一些妥协是必须的.在实践中.人们倾向于在网络的**第一个卷积层做出妥协**.例如，可以妥协可能是在第一个卷积层使用步长为2,尺寸为7x7的滤波器(比如在ZFnet中).在AlexNet中,滤波器的尺寸的11x11,步长为4.



**关于计算量上的考量请查看**[CS231n课程笔记翻译：卷积神经网络笔记](https://zhuanlan.zhihu.com/p/22038289?refer=intelligentunit)末尾.



### 12 More Conv Kernel

[1] 1x1卷积:一些论文中使用了1x1的卷积,这个方法最早是在论文[Network in Network](https://arxiv.org/pdf/1312.4400.pdf)中出现.人们刚开始看见这个1x1卷积的时候比较困惑,尤其是那些具有信号处理专业背景的人.因为信号是2维的,所以1x1卷积就没有意义.但是,在卷积神经网络中不是这样,因为这里是对3个维度进行操作,滤波器和输入数据体的深度是一样的.比如,如果输入是[32x32x3],那么1x1卷积就是在高效地进行3维点积(因为输入深度是3个通道).

[2] 扩张卷积(空洞卷积):[Fisher Yu和Vladlen Koltun](https://arxiv.org/pdf/1511.07122.pdf)的论文给卷积层引入了一个新的叫扩张(dilation)的超参数,更多详细查看[如何理解空洞卷积(dilated convolution)](https://www.zhihu.com/question/54149221)

**Ps:**

[1] [ConvNetJS CIFAR-10 demo](https://cs.stanford.edu/people/karpathy/convnetjs/demo/cifar10.html) 可以让你在服务器上实时地调试卷积神经网络的结构,观察计算结果.

[2] [State of the art ResNets in Torch7](http://torch.ch/blog/2016/02/04/resnets.html)

[3] 您可以在这个链接中了解自 2012 年以来所有获得 ImageNet 竞赛冠军的 CNN 模型（https://adeshpande3.github.io/The-9-Deep-Learning-Papers-You-Need-To-Know-About.html